In [1]:
from scipy.signal import hilbert
from scipy.signal import welch
from scipy.signal import butter, lfilter,firls
from scipy import signal
import sys
import os
# setting path
#sys.path.append('../')
#from utils import METHODS_EOS
import pandas as pd
from mne.time_frequency import psd_array_multitaper, psd_array_welch
from fooof import FOOOF
import numpy as np
import argparse
import mne.io
import mne
import multiprocessing as mp
from scipy.io import loadmat
from scipy.io import savemat
import glob
from joblib import Parallel, delayed
import multiprocessing as mp  # You can keep this if needed elsewhere


In [2]:
import pandas as pd
import numpy as np
import scipy
import neurokit2 as nk
from numpy import (absolute, angle, arctan2,
                   diff, exp, imag, mean, real, square,
                   var)
from scipy.signal import hilbert
from scipy.io import loadmat

def pcf(data):
    """Estimate the pair correlation function (PCF) in a network of
    oscillators, equivalent to the susceptibility in statistical physics.
    The PCF shows a sharp peak at a critical value of the coupling between
    oscillators, signaling the emergence of long-range correlations between
    oscillators.

    Parameters
    ----------
    data : 2d array
        The filtered input data, where the first dimension is the different
        oscillators, and the second dimension is time.

    Returns
    -------
    pcf : float
        The pair correlation function, a scalar value >= 0.
    orpa: float
        Absolute value of the order parameter (degree of synchronicity)
        averaged over time, being equal to 0 when the oscillators’ phases are
        uniformly distributed in [0,2π ) and 1 when they all have the same
        phase.
    orph_vector: 1d array (length = N_timepoints)
        Order parameter phase for every moment in time.
    orpa_vector: 1d array (length = N_timepoints)
        Absolute value of the order parameter (degree of synchronicity) for
        every moment in time.

    References
    ----------
    Yoon et al. (2015) Phys Rev E 91(3), 032814.
    """

    N_ch = min(data.shape)  # Nr of channels
    N_time = max(data.shape)  # Nr of channels

    # inifialize empty array
    inst_phase = np.zeros(data.shape)
    z_vector = []

    # calculate Phase of
    for i in range(N_ch):
        inst_phase[i,:] = np.angle(scipy.signal.hilbert(data[i,:]))

    for t in range(N_time):
        # get global synchronization order parameter z over time
        z_vector.append(np.mean(exp(1j * inst_phase[:,t])))

    z_vector = np.array(z_vector)

    #  r =|z| degree of synchronicity
    orpa_vector = abs(z_vector)
    # get order phases
    orph_vector = arctan2(imag(z_vector), real(z_vector))

    # get PCF = variance of real part of order parameter
    # var(real(x)) == (mean(square(real(x))) - square(mean(real(x))))
    pcf = N_ch * var(real(z_vector))
    # time-averaged Order Parameter
    orpa = mean(orpa_vector);

    return pcf, orpa, orph_vector, orpa_vector



In [3]:
def calculate_values(trial, epochs):

    data_tr = epochs[trial].get_data()[0]

    #PLE = Methods_EOS.ple(data_tr, m = 5, tau = 2)
    #PLI = Methods_EOS.pli(data_tr)
    PCF, OR_mean, orph_vector_tr, orpa_vector_tr = pcf(data_tr)
    print(f'done Trial {str(trial)}')

    #return PLI, PLE, PCF, OR_mean, OR_var
    return PCF, OR_mean


In [ ]:
out_dir = 'data/output/EOS/'
in_dir = 'data/input/10s/'
minfreq = 0.1
maxfreq = 40
# output
PCF_mean = []
OR_mean = []
sub_all = []
day_all = []   
condition_all = []

# make output directory
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# load patient info and conditions
paths = glob.glob(f'{in_dir}*.fif')    
paths.sort()
paths = paths[:2]

#loop over all conditions and particiants
for i, path in enumerate(paths):
    sub = os.path.basename(path)[:4]
    day = os.path.basename(path)[5:9]
    condition = os.path.basename(path)[10:-8]



    print(f"Analyzing Synchrony of {path}");

    #################################
    #          LOAD  DATA          #
    #################################

    epochs_mne = mne.read_epochs(path, preload=True)
    epochs_mne.interpolate_bads(reset_bads=True)
    epochs_mne.pick_types(eeg=True, meg=False, stim=False, eog=False, ecg=False, emg=False, misc=False, exclude='bads')

    epochs = epochs_mne.get_data()[:,:32,:]
    fs = 256 #check this


    # prepare data
    #epochs_res = epochs_mne.resample(250)
    epochs_filt = epochs_mne.filter(minfreq, maxfreq, verbose = False)

    # if data is too long only use the first 3 min of data
    nr_trials = min([len(epochs_filt),30]);
    nr_channels =  epochs_filt.info['nchan']

    ###############################################
    #    Calculate Pair Correlation Function     #
    ###############################################

    pool = mp.Pool(mp.cpu_count())
    # loop over every time segment

    # prepare input for parallel function
    input = []
    for trial in range(nr_trials):
        input.append((trial,epochs_filt))


    num_cores = mp.cpu_count()
    results = Parallel(n_jobs=num_cores)(
        delayed(calculate_values)(*args) for args in input
    )

    '''
    pool = mp.Pool(mp.cpu_count())
    results = pool.starmap(calculate_values,input)
    pool.close()
    '''
    
    PCF_mean.append(np.median(pd.DataFrame(results)[0]))
    OR_mean.append(np.median(pd.DataFrame(results)[1]))
    sub_all.append(sub)
    day_all.append(day)
    condition_all.append(condition)

    # save dataframe
    output_df = {'sub':sub_all, 'day': day_all,'condition':condition_all, 'PCF_mean':PCF_mean,'OR_mean':OR_mean}
    output_df = pd.DataFrame(output_df)
    output_df.to_csv(f'{out_dir}/EOS_{minfreq}_{maxfreq}.csv', index=False, sep=',')


# save dataframe
output_df = {'sub':sub_all, 'day': day_all,'condition':condition_all, 'PCF_mean':PCF_mean,'OR_mean':OR_mean}
output_df = pd.DataFrame(output_df)
output_df.to_csv(f'{out_dir}/EOS_{minfreq}_{maxfreq}.csv', index=False, sep=',')



Analyzing Synchrony of data/input/10s/sub0-day1-jhana-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub0-day1-jhana-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 30 sensor positions
Interpolating 2 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/687706252.py:39: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()[:,:32,:]
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/687706252.py:45: RuntimeWarning: filter_length (8449) is longer than the signal (2560), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filt = epochs_mne.filter(minfreq, maxfreq, verbose = False)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/2592650906.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/2592650906.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/fold

done Trial 0
done Trial 1
done Trial 12
done Trial 13
done Trial 3
done Trial 14
done Trial 15
done Trial 4
done Trial 2
done Trial 16
done Trial 17
done Trial 18
done Trial 19
done Trial 21
done Trial 20
done Trial 23
done Trial 22
done Trial 24
done Trial 25
done Trial 26
done Trial 27


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/2592650906.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/2592650906.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/2592650906.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/2592650906.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/2592650906.py:3: FutureWarning: The current default of copy

done Trial 28
done Trial 29
done Trial 5
done Trial 6
done Trial 7
done Trial 8
done Trial 10
done Trial 9
done Trial 11
Analyzing Synchrony of data/input/10s/sub0-day1-mindfulness-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub0-day1-mindfulness-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/2592650906.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning


Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/687706252.py:36: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  epochs_mne.interpolate_bads(reset_bads=True)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/687706252.py:39: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()[:,:32,:]
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/687706252.py:45: RuntimeWarning: filter_length (8449) is longer than the signal (2560), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filt = epochs_mne.filter(minfreq, maxfreq, verbose = False)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_36616/2592650906.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7s

done Trial 0
done Trial 1
done Trial 2
done Trial 3
done Trial 4
done Trial 5
done Trial 7
done Trial 6
done Trial 8
done Trial 9
done Trial 11
done Trial 10
done Trial 12
done Trial 13
done Trial 14
done Trial 15
done Trial 16
done Trial 17
done Trial 18
done Trial 19
done Trial 20
done Trial 21
done Trial 22
done Trial 23
done Trial 24
done Trial 25
done Trial 27
done Trial 26
done Trial 28
done Trial 29
